# Working with ISIC Images
Images are at the heart of the ISIC Archive, and this notebook
will give you a sense of what the ```isicarchive``` package
can do in terms of retrieving images from the archive, and the
kinds of methods you can apply to them.
## Preparation
We'll start again by creating an ```IsicApi``` object, which
allows us to access the various pieces of information and images
stored in the archive. The call to ```cache_images()``` will
then download information (and meta information) about all
images that are available--if you are logged in as a regular
user, you may have access to additional images! **If you're
using the ```cache_images()``` method, please use a different
cache folder if you want to work with both publicly available
and private images in separate sessions.**

Next, we'll select all images from the ```IsicApi``` object that
have a specific diagnosis, and then display those images.

In [1]:
# Settings (please adapt to your needs!)
username = 'weberj3@mskcc.org'

# DO NOT PUT A PASSWORD into the document, it will be requested,
# or you can store it in a privately readable ~/.netrc file!

cache_folder = 'C:\\Users\\weberj3\\Documents\\ISIC'
# cache_folder = '/Data/MSKCC/isicarchive/cache'

# setting debug flag, this will print out every web-API operation
debug = True

# Preparation
from isicarchive.api import IsicApi
from isicarchive import func, imfunc
api = IsicApi(username, cache_folder=cache_folder, debug=debug)

Requesting (auth) https://isic-archive.com/api/v1/image/histogram
Retrieved dict with 25 keys.
Requesting (auth) https://isic-archive.com/api/v1/dataset with params: {'limit': 0, 'detail': 'true'}
Retrieved list with 29 items.
Requesting (auth) https://isic-archive.com/api/v1/study with params: {'limit': 0, 'detail': 'true'}
Retrieved list with 18 items.


In [ ]:
# caching images and segmentations; this can take a long time!!
# so you may want to skip this cell for now...
api.cache_images()
api.cache_segmentations()

In [ ]:
# list studies
api.studies

In [3]:
# load a study and cache the images associated with it
study = api.study('ISIC Annotation Study - All Features')

Requesting (auth) https://isic-archive.com/api/v1/annotation with params: {'studyId': '5a32cde91165975cf58a469c', 'detail': 'true'}
Retrieved list with 840 items.


In [ ]:
# load image and data
image = api.image(study.images[0])
image.load_imagedata()
image.load_superpixels()
image.map_superpixels()

In [ ]:
# create superpixel outlines as SVG paths
superpixel_colors = func.superpixel_colors()
outline_svg = imfunc.superpixel_outlines(
    image.superpixels['map'],
    image.superpixels['idx'].shape,
    out_format='osvg',
    path_attribs=superpixel_colors)
outlines = imfunc.superpixel_outlines(
    image.superpixels['map'],
    image.superpixels['idx'].shape,
    out_format='osvgp',
    path_attribs=superpixel_colors)

# display one path
outlines[0]

In [ ]:
# demostrate how to color a single superpixel
image_data = image.data
image_shape = image_data.shape
image_data.shape = (image_shape[0] * image_shape[1], -1)
map = image.superpixels['map']
superpixel_index = 472
pixel_count = map[superpixel_index, -1]
superpixel_pixels = map[superpixel_index, 0:pixel_count]
image_data[superpixel_pixels, 0] = 255
image_data[superpixel_pixels, 1] = 0
image_data[superpixel_pixels, 2] = 0
image_data.shape = image_shape

import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(image_data)
plt.show()

In [ ]:
segmentation = api.segmentation('ISIC_0023877')

In [ ]:
study_features = []
study.features

In [4]:
completed_annotations = {}
for a in study.annotations:
    if not 'markups' in a:
        continue
    for (f,m) in a['markups'].items():
        if not m:
            continue
        if not f in completed_annotations:
            completed_annotations[f] = 1
        else:
            completed_annotations[f] += 1

In [2]:
api._feature_colors

{'Circles & Semicircles : NOS': [120, 120, 120],
 'Circles & Semicircles : Brown': [184, 136, 112],
 'Circles & Semicircles : Circle within a circle': [160, 128, 120],
 'Circles & Semicircles : White': [192, 192, 192],
 'Dots : NOS': [48, 48, 48],
 'Dots : Black': [48, 40, 24],
 'Dots : Blue-gray': [8, 64, 128],
 'Dots : Brown': [76, 48, 40],
 'Dots : Irregular': [96, 56, 56],
 'Dots : Regular': [72, 48, 48],
 'Globules / Clods : NOS': [128, 96, 48],
 'Globules / Clods : Blue': [16, 48, 96],
 'Globules / Clods : Blue-gray ovoid nests': [32, 56, 112],
 'Globules / Clods : Cobblestone pattern': [48, 48, 104],
 'Globules / Clods : Comedo-like openings': [192, 136, 112],
 'Globules / Clods : Concentric': [184, 120, 88],
 'Globules / Clods : Irregular': [144, 64, 120],
 'Globules / Clods : Lacunae : Red': [144, 24, 16],
 'Globules / Clods : Lacunae : Red-purple': [144, 16, 80],
 'Globules / Clods : Shiny white blotches and strands': [192, 136, 112],
 'Lines : NOS': [40, 40, 64],
 'Lines : A

In [5]:
sorted_features = {}
for k in sorted(completed_annotations.keys()):
    sorted_features[k] = completed_annotations[k]
sorted_features

{'Circles & Semicircles : Brown': 6,
 'Circles & Semicircles : Circle within a circle': 4,
 'Circles & Semicircles : NOS': 1,
 'Circles & Semicircles : White': 1,
 'Dots : Black': 11,
 'Dots : Blue-gray': 33,
 'Dots : Brown': 26,
 'Dots : Irregular': 39,
 'Dots : NOS': 2,
 'Dots : Regular': 14,
 'Globules / Clods : Blue': 23,
 'Globules / Clods : Blue-gray ovoid nests': 43,
 'Globules / Clods : Cobblestone pattern': 16,
 'Globules / Clods : Comedo-like openings': 71,
 'Globules / Clods : Concentric': 14,
 'Globules / Clods : Irregular': 37,
 'Globules / Clods : Lacunae : Black': 4,
 'Globules / Clods : Lacunae : Red': 36,
 'Globules / Clods : Lacunae : Red-purple': 19,
 'Globules / Clods : Leaflike area': 24,
 'Globules / Clods : Milia-like cysts': 88,
 'Globules / Clods : Milky red': 12,
 'Globules / Clods : NOS': 10,
 'Globules / Clods : Regular': 23,
 'Globules / Clods : Rim of brown globules': 11,
 'Globules / Clods : White': 1,
 'Lines : Angulated lines / Polygons / Zig-zag patter